# Gold Layer Processing - Worldwide Earthquake Events API 

In [1]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
# ensure the below library is installed on your fabric environment
import reverse_geocoder as rg

StatementMeta(, 3ba247a2-f92a-4b51-a431-bdeb4b6640e6, 5, Finished, Available, Finished)

In [2]:
df = spark.read.table("earthquake_events_silver").filter(col('time') > start_date)

StatementMeta(, 3ba247a2-f92a-4b51-a431-bdeb4b6640e6, 6, Finished, Available, Finished)

In [6]:
coordinates = (25.4405,142.8273)
rg.search(coordinates)

StatementMeta(, 3ba247a2-f92a-4b51-a431-bdeb4b6640e6, 10, Finished, Available, Finished)

[{'lat': '34.98333',
  'lon': '139.86667',
  'name': 'Tateyama',
  'admin1': 'Chiba',
  'admin2': '',
  'cc': 'JP'}]

In [3]:
display(df)

StatementMeta(, 3ba247a2-f92a-4b51-a431-bdeb4b6640e6, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 573abb1c-7dbf-4a7c-8aab-a5ef5bb145f2)

In [7]:
def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """
    coordinates = (float(lat), float(lon))
    return rg.search(coordinates)[0].get('cc')

StatementMeta(, 3ba247a2-f92a-4b51-a431-bdeb4b6640e6, 11, Finished, Available, Finished)

In [8]:
# registering the udfs so they can be used on spark dataframes
get_country_code_udf = udf(get_country_code, StringType())

StatementMeta(, 3ba247a2-f92a-4b51-a431-bdeb4b6640e6, 12, Finished, Available, Finished)

In [9]:
# adding country_code and city attributes
df_with_location = \
                df.\
                    withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))

StatementMeta(, 3ba247a2-f92a-4b51-a431-bdeb4b6640e6, 13, Finished, Available, Finished)

In [ ]:
display(df_with_location)

In [11]:
# adding significance classification
df_with_location_sig_class = \
                            df_with_location.\
                                withColumn('sig_class', 
                                            when(col("sig") < 100, "Low").\
                                            when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                            otherwise("High")
                                            )

StatementMeta(, 3ba247a2-f92a-4b51-a431-bdeb4b6640e6, 15, Finished, Available, Finished)

In [13]:
# appending the data to the gold table
df_with_location_sig_class.write.mode('append').saveAsTable('earthquake_events_gold')

StatementMeta(, 3ba247a2-f92a-4b51-a431-bdeb4b6640e6, 17, Finished, Available, Finished)